In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
import  pyspark.sql.functions as fun
import numpy as np
from pyspark.sql.types import *

In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
sc = spark.sparkContext

Create an RDD from a list of numbers (1,50) using numpy methods

In [61]:
numbers = np.arange(1, 51).tolist()
rdd = sc.parallelize(numbers)
print(rdd.collect())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]


Find the sum, average, maximum, minimum, and count

In [62]:
#Sum
rdd.sum()

1275

In [63]:
#avg
rdd.mean()

25.5

In [64]:
#Count
rdd.count()

50

In [65]:
#Min
rdd.min()

1

In [66]:
#Max
rdd.max()

50

Count how many numbers are even vs. odd.

In [81]:
rdd1 = rdd
even_count = rdd.filter(lambda x: x % 2 == 0).count()
odd_count= rdd1.count() - even_count
print(f" odd count: {odd_count} even count:{even_count}")

 odd count: 25 even count:25


You have the following data of people info ('Name', 'Age'), answer the following questions

In [82]:
people_data = [("Nada ", 25), ("Mona", 30), ("Ahmed", 35), ("Khaled", 40),("Ahmed", 35), ('Nada ', 25)]
rdd_people = sc.parallelize(people_data)
rdd_people.collect()

[('Nada ', 25),
 ('Mona', 30),
 ('Ahmed', 35),
 ('Khaled', 40),
 ('Ahmed', 35),
 ('Nada ', 25)]

Find the oldest person

In [83]:
rdd_people.max(lambda x: x[1])

('Khaled', 40)

Compute the average age

In [84]:
rdd_people.map(lambda x: x[1]).mean()

31.666666666666668

Group all the names by their age

In [108]:
rdd_people.groupBy(lambda x: x[1]).mapValues(list).collect()

[(30, [('Mona', 30)]),
 (40, [('Khaled', 40)]),
 (25, [('Nada ', 25), ('Nada ', 25)]),
 (35, [('Ahmed', 35), ('Ahmed', 35)])]

Take the following text and put it in a text file named russia.txt and load it into rdd

"Russia is the largest country in the world by land area
Moscow is the capital city of Russia
The Russian language is one of the most widely spoken languages in the world
Russia is known for its rich history and culture
The Trans-Siberian Railway is the longest railway line in the world
Russia has a strong tradition in literature, music and ballet
The country is famous for its cold winters and vast landscapes
Russia is a major player in global energy production
"

In [90]:
rdd_russia = sc.textFile("Russia.txt")
rdd_russia.collect()

['Russia is the largest country in the world by land area',
 'Moscow is the capital city of Russia',
 'The Russian language is one of the most widely spoken languages in the world',
 'Russia is known for its rich history and culture ',
 'The Trans-Siberian Railway is the longest railway line in the world',
 'Russia has a strong tradition in literature, music and ballet ',
 'The country is famous for its cold winters and vast landscapes ',
 'Russia is a major player in global energy production']

Count the total number of lines.

In [91]:
rdd_russia.count()

8

Count how many lines contain the word "Russia"

In [92]:
rdd_russia.filter(lambda line: "Russia" in line).count()

6

Find the most 5 frequent word in the file.

In [93]:
words = rdd_russia.flatMap(lambda line: line.split(" "))
word_counts = words.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
sorted_word_counts = word_counts.sortBy(lambda x: x[1], ascending=False)
sorted_word_counts.take(5)

[('is', 7), ('the', 7), ('Russia', 5), ('in', 5), ('world', 3)]

Tokenize words

In [94]:
tokenized_words = rdd_russia.flatMap(lambda line: line.lower().split(" "))
print(tokenized_words.collect())

['russia', 'is', 'the', 'largest', 'country', 'in', 'the', 'world', 'by', 'land', 'area', 'moscow', 'is', 'the', 'capital', 'city', 'of', 'russia', 'the', 'russian', 'language', 'is', 'one', 'of', 'the', 'most', 'widely', 'spoken', 'languages', 'in', 'the', 'world', 'russia', 'is', 'known', 'for', 'its', 'rich', 'history', 'and', 'culture', '', 'the', 'trans-siberian', 'railway', 'is', 'the', 'longest', 'railway', 'line', 'in', 'the', 'world', 'russia', 'has', 'a', 'strong', 'tradition', 'in', 'literature,', 'music', 'and', 'ballet', '', 'the', 'country', 'is', 'famous', 'for', 'its', 'cold', 'winters', 'and', 'vast', 'landscapes', '', 'russia', 'is', 'a', 'major', 'player', 'in', 'global', 'energy', 'production']


Remove stopwords (a, the, is, to, in, of).

In [95]:
stopwords = ['a', 'the', 'is', 'to', 'in', 'of']
filtered_words = tokenized_words.filter(lambda word: word not in stopwords)
print(filtered_words.collect())

['russia', 'largest', 'country', 'world', 'by', 'land', 'area', 'moscow', 'capital', 'city', 'russia', 'russian', 'language', 'one', 'most', 'widely', 'spoken', 'languages', 'world', 'russia', 'known', 'for', 'its', 'rich', 'history', 'and', 'culture', '', 'trans-siberian', 'railway', 'longest', 'railway', 'line', 'world', 'russia', 'has', 'strong', 'tradition', 'literature,', 'music', 'and', 'ballet', '', 'country', 'famous', 'for', 'its', 'cold', 'winters', 'and', 'vast', 'landscapes', '', 'russia', 'major', 'player', 'global', 'energy', 'production']


Count the frequency of each word

In [96]:
word_freq = filtered_words.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
print(word_freq.collect())

[('largest', 1), ('country', 2), ('world', 3), ('by', 1), ('land', 1), ('area', 1), ('capital', 1), ('russian', 1), ('language', 1), ('most', 1), ('widely', 1), ('known', 1), ('for', 2), ('history', 1), ('and', 3), ('', 3), ('line', 1), ('literature,', 1), ('music', 1), ('famous', 1), ('cold', 1), ('winters', 1), ('landscapes', 1), ('player', 1), ('energy', 1), ('production', 1), ('russia', 5), ('moscow', 1), ('city', 1), ('one', 1), ('spoken', 1), ('languages', 1), ('its', 2), ('rich', 1), ('culture', 1), ('trans-siberian', 1), ('railway', 2), ('longest', 1), ('has', 1), ('strong', 1), ('tradition', 1), ('ballet', 1), ('vast', 1), ('major', 1), ('global', 1)]


Sql Schema

In [97]:
schema = 'id integer, name string, age integer, salary integer'
data = [
    (1, "Ali", 25, 4000),
    (2, "Mariam", 30, 6000),
    (3, "Omar", 35, 7000),
    (4, "Sara", 28, 5000),
    (5, "Omar", 25, 6500),
    (6, "Mariam", 26, 7500)
]

df = spark.createDataFrame(data,schema)

Show schema and first 2 rows

In [98]:
df.printSchema()
df.show(2)

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: integer (nullable = true)

+---+------+---+------+
| id|  name|age|salary|
+---+------+---+------+
|  1|   Ali| 25|  4000|
|  2|Mariam| 30|  6000|
+---+------+---+------+
only showing top 2 rows



Select only name and salary

In [99]:
df.select("name", "salary").show()

+------+------+
|  name|salary|
+------+------+
|   Ali|  4000|
|Mariam|  6000|
|  Omar|  7000|
|  Sara|  5000|
|  Omar|  6500|
|Mariam|  7500|
+------+------+



Find the average salary

In [100]:
df.select(fun.avg("salary")).show()

+-----------+
|avg(salary)|
+-----------+
|     6000.0|
+-----------+



Filter employees older than 28

In [101]:
df.filter(df['age'] > 28).show()

+---+------+---+------+
| id|  name|age|salary|
+---+------+---+------+
|  2|Mariam| 30|  6000|
|  3|  Omar| 35|  7000|
+---+------+---+------+



Count distinct values in the name column

In [102]:
df.select("name").distinct().count()

4

Group by a the name column and find average salary

In [103]:
df.groupBy("name").agg(fun.avg("salary").alias("Average")).show()

+------+-------+
|  name|Average|
+------+-------+
|  Omar| 6750.0|
|Mariam| 6750.0|
|   Ali| 4000.0|
|  Sara| 5000.0|
+------+-------+



In [104]:
# Creating the file first
null_data_content = """Id,Name,Sales
emp1,John,
emp2,,
emp3,,345.0
emp4,Cindy,456.0"""
with open("NullData.csv", "w") as f:
    f.write(null_data_content)

df1 = spark.read.csv("NullData.csv", header=True, inferSchema=True)
df1.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| NULL|
|emp2| NULL| NULL|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



Find the avg sales

In [105]:
avg_sales = df1.select(fun.avg("Sales")).collect()[0][0]
print(avg_sales)

400.5


Replace null name with 'Unknown' and sales with the avg sales of the column

In [106]:
df1.na.fill({'Name': 'Unknown', 'Sales': avg_sales}).show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John|400.5|
|emp2|Unknown|400.5|
|emp3|Unknown|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+

